In [3]:
# Imports (External)
import numpy as np
import pandas as pd
import datetime
from datetime import datetime

from monthdelta import monthdelta
import xlrd
import xlsxwriter
from collections import OrderedDict
import pickle

import sys
sys.path.append('../')  

# Visualization/plotting imports
import matplotlib as mpl
import matplotlib.pyplot as plt

# Machine learning imports
import pywt
from pywt import wavedec, waverec
from scipy import signal
from statsmodels.robust import mad

import sklearn
import tensorflow as tf
import keras
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import optimizers
from keras import regularizers

# Internal Imports
from wsae_lstm.utils import pickle_load

In [4]:
dict_dataframes_index=pickle_load(path_filename="../data/interim/clean_data_index_interval.pickle")
#dict_dataframes_index.keys()

In [5]:
#dict_dataframes_index['csi300 index data'][1].drop(['date'],axis=1,inplace=True)

In [6]:
#dict_dataframes_index['csi300 index data'][1].reset_index(drop=True,inplace=True)

In [7]:
dict_dataframes_index['csi300 index data'][1].head()

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08
2008-07-03,2654.48,2807.68,2617.26,2760.61,456603.05,72.73,2.5762,-206.941406,-112.113057,190.424,3109.044731,2978.942155,2820.8364,-208.925,2753.2792,-82.064,-2.886857,-0.031251,-5.855737e+07
2008-07-04,2751.21,2783.85,2716.02,2741.85,379050.10,72.71,2.5632,-200.759162,-81.997539,67.825,3073.107115,2956.362140,2810.0548,-239.055,2738.4454,-249.421,-8.338289,-0.017237,-7.467503e+07
2008-07-07,2747.61,2890.99,2747.61,2882.76,527320.24,72.71,2.5679,-182.386907,21.707767,149.133,3046.256923,2949.352699,2819.3370,66.742,2756.6342,109.687,3.955427,0.010701,-3.531151e+07


In [8]:
dict_dataframes_index['csi300 index data'][1].head()

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08
2008-07-03,2654.48,2807.68,2617.26,2760.61,456603.05,72.73,2.5762,-206.941406,-112.113057,190.424,3109.044731,2978.942155,2820.8364,-208.925,2753.2792,-82.064,-2.886857,-0.031251,-5.855737e+07
2008-07-04,2751.21,2783.85,2716.02,2741.85,379050.10,72.71,2.5632,-200.759162,-81.997539,67.825,3073.107115,2956.362140,2810.0548,-239.055,2738.4454,-249.421,-8.338289,-0.017237,-7.467503e+07
2008-07-07,2747.61,2890.99,2747.61,2882.76,527320.24,72.71,2.5679,-182.386907,21.707767,149.133,3046.256923,2949.352699,2819.3370,66.742,2756.6342,109.687,3.955427,0.010701,-3.531151e+07


In [9]:
len(dict_dataframes_index['csi300 index data'][1])

612

In [10]:
len(dict_dataframes_index['csi300 index data'][1].columns)

19

In [11]:
len(dict_dataframes_index['csi300 index data'])

24

In [12]:
# train
dict_dataframes_index['csi300 index data'][1].index[0]

Timestamp('2008-07-01 00:00:00')

In [13]:
# validate
dict_dataframes_index['csi300 index data'][1].index[0] + monthdelta(24) 

Timestamp('2010-07-01 00:00:00')

In [14]:
# test
dict_dataframes_index['csi300 index data'][1].index[0] + monthdelta(27) 

Timestamp('2010-10-01 00:00:00')

In [165]:
def tvt_split(df):
    train = df.index[0]
    validate = df.index[0] + monthdelta(24) 
    test = df.index[0] + monthdelta(27) 
    test_end = df.index[0] + monthdelta(30) 

#     print("Train start:\n",train)
#     print("Train end/validate start:\n",validate)
#     print("Validate end/test start:\n",test)
#     print("Test end:\n",test_end)
    
    df_train = pd.DataFrame(df[(df.index >= train) & (df.index <= validate)])
    df_validate = pd.DataFrame(df[(df.index >= validate) & (df.index <= test)])
    df_test = pd.DataFrame(df[(df.index >= test) & (df.index <= test_end)])
    return df_train,df_validate,df_test

In [166]:
#df_train,df_validate,df_test = tvt_split(dict_dataframes_index['csi300 index data'][1])

In [167]:
def dict_df_tvt_split(df):
    dict_dataframes = {}
    split_count = 24
    month_increment = 0
    interval_index = 1
    while split_count > 0:
        df_train, df_validate, df_test = tvt_split(df[interval_index])
        split_count -= 1
        interval_index += 1
        dict_dataframes[interval_index] = df_train, df_validate, df_test
    return dict_dataframes

In [168]:
dict_df_tvt = dict_df_tvt_split(dict_dataframes_index['csi300 index data'])

In [169]:
len(dict_df_tvt)

24

In [209]:
dict_dataframes = {}
for key in dict_dataframes_index:
    print(key)
    dict_dataframes[key] = dict_df_tvt_split(dict_dataframes_index[key])

csi300 index data
nifty 50 index data
hangseng index data
nikkei 225 index data
s&p500 index data
djia index data


In [210]:
len(dict_dataframes)

6

In [211]:
len(dict_dataframes['csi300 index data'])

24

In [212]:
type(dict_dataframes['csi300 index data'])

dict

In [213]:
dict_dataframes['csi300 index data'].keys()

dict_keys([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])

In [ ]:
dict_dataframes['csi300 index data'] = {k-1: v for k, v in dict_dataframes['csi300 index data'].items()}

In [217]:
dict_dataframes['csi300 index data'].keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])

In [226]:
len(dict_dataframes['csi300 index data'][1][1])

63

In [232]:
dict_dataframes['csi300 index data'][1][0].head(1)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.2,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.2,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.77,-9.424605,-0.097927,-8.726278e+07


In [ ]:
dict_dataframes['csi300 index data'][1][1].head(1)

In [233]:
dict_dataframes['csi300 index data'][1][2].head(1)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2010-10-08,2965.65,3055.84,2963.24,3044.23,999947.2,77.2,1.7325,14.071181,187.594016,120.262,2912.040962,2913.539251,2898.0211,194.405,2928.111,111.679,3.80825,0.043045,1.079573e+08


In [145]:
# def dict_df_tvt_split_test(dict_dataframes):
#     subdict_dataframes = {}
#     split_count = 24
#     interval_index = 1
#     for dataframe in dict_dataframes:
#         subdict_dataframes[dataframe][interval_index] = dict_tvt_split(dict_dataframes[dataframe])
#         interval_index += 1
#     return subdict_dataframes

In [53]:
#train.head(2)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2014-04-01,2143.72,2170.10,2141.15,2163.12,488664.03,80.08,2.866,-11.448999,45.062472,28.944,2150.219000,2158.446226,2150.5768,-13.439,2157.6278,40.279,1.897415,0.019530,7.622596e+07
2014-04-02,2166.07,2182.68,2164.01,2180.73,650392.57,80.23,2.950,-8.435674,93.409788,19.568,2151.096615,2160.568204,2156.5625,6.287,2159.5638,37.689,1.758672,0.042995,1.775670e+08


In [54]:
#validate.head(2)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2016-04-01,3213.67,3222.61,3165.86,3221.90,1181221.39,94.5626,2.014,35.386273,40.950404,56.7569,3109.797608,3154.620636,3205.23836,40.0441,3192.27922,131.8620,4.267333,-0.008476,8.859940e+08
2016-04-05,3211.31,3271.93,3205.21,3264.49,1460477.72,94.6133,1.993,39.489875,108.008262,66.7236,3121.968962,3165.083985,3206.74322,66.6689,3211.23072,140.2819,4.490165,0.011853,9.366479e+08


In [55]:
#test.head(2)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2016-07-01,3156.93,3170.26,3148.30,3154.2,717462.83,95.6655,2.027,5.204202,141.961406,21.9635,3122.562035,3126.451547,3126.38819,36.8835,3143.29134,37.8321,1.213980,0.013165,3.038717e+08
2016-07-04,3136.39,3210.60,3134.02,3204.7,1333105.29,95.5253,2.014,10.705219,180.887755,76.5783,3128.157046,3133.903419,3135.59037,127.5408,3160.12172,110.0214,3.555185,0.030698,4.150349e+08
